<a href="https://colab.research.google.com/github/PeepeR19/CSCI167_PyTorch_Notebook/blob/main/ComputerScienceDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 0. Setup: installs and Kaggle credentials
!pip install -q kaggle datasets torch torchvision

import os
from pathlib import Path

# --- Kaggle credentials (one-time per Colab runtime) ---
# 1. In Colab go to: Files > Upload, and upload your kaggle.json
# 2. Then run this cell.

if not Path("kaggle.json").exists():
    print(">>> Upload your kaggle.json in the Files panel or via `files.upload()`.")
else:
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("Kaggle credentials set up.")

Kaggle credentials set up.


In [ ]:
  #@title 1. Download Face dataset from Kaggle + set paths

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

# --- Kaggle face dataset (Vasuki Patel) ---
# Dataset description: 2562 images, 31 classes (celebrities). :contentReference[oaicite:3]{index=3}
KAGGLE_DATASET = "vasukipatel/face-recognition-dataset"
KAGGLE_ZIP = DATA_DIR / "face-recognition-dataset.zip"
FACES_ROOT = DATA_DIR / "face-recognition-dataset"  # we will unzip here

if not FACES_ROOT.exists():
    if not KAGGLE_ZIP.exists():
        !kaggle datasets download -d $KAGGLE_DATASET -p $DATA_DIR
    !unzip -q $KAGGLE_ZIP -d $FACES_ROOT

print("Kaggle face dataset downloaded. Inspect directories in the left panel.")
print("If needed, adjust FACES_ROOT to point at the folder that contains per-person subfolders.")

Dataset URL: https://www.kaggle.com/datasets/vasukipatel/face-recognition-dataset
License(s): CC0-1.0
 94% 684M/726M [00:10<00:00, 76.6MB/s]
100% 726M/726M [00:10<00:00, 71.0MB/s]
Kaggle face dataset downloaded. Inspect directories in the left panel.
If needed, adjust FACES_ROOT to point at the folder that contains per-person subfolders.


In [ ]:
#@title 1b. (Alternate) Download Hugging Face face dataset (bigger, 105 classes)

from huggingface_hub import snapshot_download

USE_HF_FACE_DATASET = False  # <-- set to True if you want HF instead of Kaggle

HF_FACE_REPO = "AI-Solutions-KK/face_recognition_dataset"
HF_FACE_LOCAL = DATA_DIR / "hf_face_recognition_dataset"

if USE_HF_FACE_DATASET:
    snapshot_download(
        repo_id=HF_FACE_REPO,
        repo_type="dataset",
        local_dir=HF_FACE_LOCAL,
        local_dir_use_symlinks=False,
    )
    print("Hugging Face face dataset downloaded to:", HF_FACE_LOCAL)

# This dataset has ~18k+ images and 105 identities in a folder-per-person structure. :contentReference[oaicite:5]{index=5}

In [ ]:
#@title 2. Imports + device + transforms

import random
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Dataset

import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder, MNIST
from torchvision.models import resnet18, ResNet18_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --- Shared transforms for faces (Kaggle or HF) ---
IMG_SIZE = 224  # works with ResNet18

face_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   # ImageNet-like normalization
        std=[0.229, 0.224, 0.225]
    ),
])

# --- Transforms for MNIST (1-channel, 28x28) ---
mnist_transform = transforms.Compose([
    transforms.ToTensor(),  # [0,1]
    transforms.Normalize((0.1307,), (0.3081,)),  # standard MNIST stats
])

Using device: cpu


In [ ]:
#@title 3.1 HF Face dataset wrapper (used if USE_HF_FACE_DATASET=True)
from datasets import load_dataset

class HFFaceDataset(Dataset):
    """
    Wraps AI-Solutions-KK/face_recognition_dataset as a PyTorch Dataset.
    Each example has keys: 'image' (PIL.Image) and 'label' (int in [0, num_classes-1]).
    """
    def __init__(self, split="train", transform=None):
        self.ds = load_dataset(HF_FACE_REPO, split=split)
        self.transform = transform

        # HF already gives 'label' as integer with 105 classes; we keep as-is. :contentReference[oaicite:6]{index=6}
        self.num_classes = len(self.ds.features["label"].names)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        ex = self.ds[idx]
        img = ex["image"]
        label = ex["label"]
        if self.transform:
            img = self.transform(img)
        return img, label

In [ ]:
#@title 3.2 Face dataloaders (Kaggle or HF)
def get_face_dataloaders(
    batch_size=32,
    val_split=0.2,
    use_hf=USE_HF_FACE_DATASET,
):
    if use_hf:
        full_dataset = HFFaceDataset(split="train", transform=face_transform)
        num_classes = full_dataset.num_classes
        print(f"Using Hugging Face face dataset with {num_classes} classes.")
    else:
        # Adjust FACES_ROOT if necessary after inspecting directory tree
        global FACES_ROOT
        if not (FACES_ROOT.exists() and any(FACES_ROOT.iterdir())):
            raise RuntimeError(
                f"FACES_ROOT '{FACES_ROOT}' seems empty. "
                "Open the left file explorer and set FACES_ROOT to the folder with per-person subfolders."
            )
        full_dataset = ImageFolder(root=FACES_ROOT, transform=face_transform)
        num_classes = len(full_dataset.classes)
        print(f"Using Kaggle face dataset with {num_classes} classes.")
        print("Example classes:", full_dataset.classes[:5], "...")

    n_total = len(full_dataset)
    n_val = int(val_split * n_total)
    n_train = n_total - n_val

    train_ds, val_ds = random_split(
        full_dataset,
        [n_train, n_val],
        generator=torch.Generator().manual_seed(42),
    )

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    return train_loader, val_loader, num_classes

In [ ]:
#@title 3.3 MNIST dataloaders (for CNN benchmark)

def get_mnist_dataloaders(batch_size=64):
    """
    Standard MNIST: 60k train, 10k test, 10 classes. :contentReference[oaicite:7]{index=7}
    We'll treat the official test split as our validation set.
    """
    train_ds = MNIST(root=DATA_DIR, train=True, download=True, transform=mnist_transform)
    val_ds   = MNIST(root=DATA_DIR, train=False, download=True, transform=mnist_transform)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    num_classes = 10
    return train_loader, val_loader, num_classes

In [ ]:
#@title 4.1 Simple CNN (used in multiple experiments)
class SimpleCNN(nn.Module):
    def __init__(self, in_channels: int, num_classes: int):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),      # /2

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),      # /4

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1)),  # global average
        )
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [ ]:
#@title 4.2 ResNet18 with custom head (for faces only)

def make_resnet18_head(num_classes: int, freeze_backbone: bool = False):
    weights = ResNet18_Weights.DEFAULT
    model = resnet18(weights=weights)
    if freeze_backbone:
        for p in model.parameters():
            p.requires_grad = False
    # Replace final FC layer
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

In [ ]:
#@title 5. Training & evaluation utilities

def make_optimizer(name: str, model: nn.Module, lr: float):
    name = name.lower()
    if name == "sgd":
        return torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    elif name == "adam":
        return torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    elif name == "adamw":
        return torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
    else:
        raise ValueError(f"Unknown optimizer {name}")


def accuracy_from_logits(logits, targets):
    preds = logits.argmax(dim=1)
    correct = (preds == targets).sum().item()
    total = targets.size(0)
    return correct, total


def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        c, t = accuracy_from_logits(outputs, labels)
        correct += c
        total += t

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            c, t = accuracy_from_logits(outputs, labels)
            correct += c
            total += t

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def run_experiment(
    model,
    train_loader,
    val_loader,
    optimizer_name="sgd",
    lr=1e-3,
    epochs=5,
    experiment_name="exp",
):
    model = model.to(device)
    optimizer = make_optimizer(optimizer_name, model, lr)
    criterion = nn.CrossEntropyLoss()

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": [],
    }

    print(f"\n=== {experiment_name} ===")
    print(f"Optimizer: {optimizer_name}, lr={lr}, epochs={epochs}")

    for epoch in range(1, epochs + 1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)

        history["train_loss"].append(tr_loss)
        history["train_acc"].append(tr_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(
            f"Epoch {epoch:02d} | "
            f"train_loss={tr_loss:.4f}, train_acc={tr_acc*100:.2f}% | "
            f"val_loss={val_loss:.4f}, val_acc={val_acc*100:.2f}%"
        )

    return model, history

In [ ]:
#@title 6. Run 3 face-recognition experiments

# Get face dataloaders
face_train_loader, face_val_loader, face_num_classes = get_face_dataloaders(
    batch_size=32,
    val_split=0.2,
    use_hf=USE_HF_FACE_DATASET,
)

# --- Method 1: Simple CNN + SGD ---
face_cnn_sgd = SimpleCNN(in_channels=3, num_classes=face_num_classes)
face_cnn_sgd, hist_cnn_sgd = run_experiment(
    model=face_cnn_sgd,
    train_loader=face_train_loader,
    val_loader=face_val_loader,
    optimizer_name="sgd",
    lr=0.01,
    epochs=5,
    experiment_name="Faces - SimpleCNN + SGD",
)

# --- Method 2: Simple CNN + Adam ---
face_cnn_adam = SimpleCNN(in_channels=3, num_classes=face_num_classes)
face_cnn_adam, hist_cnn_adam = run_experiment(
    model=face_cnn_adam,
    train_loader=face_train_loader,
    val_loader=face_val_loader,
    optimizer_name="adam",
    lr=1e-3,
    epochs=5,
    experiment_name="Faces - SimpleCNN + Adam",
)

# --- Method 3: ResNet18 + Adam (transfer learning) ---
face_resnet = make_resnet18_head(num_classes=face_num_classes, freeze_backbone=False)
face_resnet, hist_resnet = run_experiment(
    model=face_resnet,
    train_loader=face_train_loader,
    val_loader=face_val_loader,
    optimizer_name="adam",
    lr=1e-4,  # usually smaller LR for pretrained models
    epochs=5,
    experiment_name="Faces - ResNet18 + Adam",
)

Using Kaggle face dataset with 2 classes.
Example classes: ['Faces', 'Original Images'] ...

=== Faces - SimpleCNN + SGD ===
Optimizer: sgd, lr=0.01, epochs=5


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 01 | train_loss=0.1881, train_acc=93.24% | val_loss=0.1117, val_acc=96.39%
Epoch 02 | train_loss=0.1200, train_acc=95.54% | val_loss=0.0880, val_acc=96.68%
Epoch 03 | train_loss=0.0938, train_acc=96.66% | val_loss=0.0691, val_acc=97.17%
Epoch 04 | train_loss=0.0994, train_acc=96.27% | val_loss=0.1237, val_acc=95.41%
Epoch 05 | train_loss=0.1032, train_acc=96.20% | val_loss=0.0505, val_acc=98.34%

=== Faces - SimpleCNN + Adam ===
Optimizer: adam, lr=0.001, epochs=5
Epoch 01 | train_loss=0.1718, train_acc=93.41% | val_loss=0.0859, val_acc=97.56%
Epoch 02 | train_loss=0.1046, train_acc=96.49% | val_loss=0.0831, val_acc=97.46%
Epoch 03 | train_loss=0.0838, train_acc=96.88% | val_loss=0.0495, val_acc=98.05%
Epoch 04 | train_loss=0.0688, train_acc=97.71% | val_loss=0.0570, val_acc=98.14%
Epoch 05 | train_loss=0.0689, train_acc=97.80% | val_loss=0.0476, val_acc=98.73%
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-

100%|██████████| 44.7M/44.7M [00:00<00:00, 54.9MB/s]



=== Faces - ResNet18 + Adam ===
Optimizer: adam, lr=0.0001, epochs=5
Epoch 01 | train_loss=0.0182, train_acc=99.20% | val_loss=0.0001, val_acc=100.00%
Epoch 02 | train_loss=0.0046, train_acc=99.95% | val_loss=0.0001, val_acc=100.00%
Epoch 03 | train_loss=0.0030, train_acc=99.95% | val_loss=0.0005, val_acc=100.00%
Epoch 04 | train_loss=0.0003, train_acc=100.00% | val_loss=0.0000, val_acc=100.00%
Epoch 05 | train_loss=0.0001, train_acc=100.00% | val_loss=0.0000, val_acc=100.00%


In [ ]:
#@title 7. MNIST benchmark with SimpleCNN + Adam

mnist_train_loader, mnist_val_loader, mnist_num_classes = get_mnist_dataloaders(batch_size=128)

# Note: MNIST is 1-channel (grayscale), 28x28.
mnist_cnn = SimpleCNN(in_channels=1, num_classes=mnist_num_classes)

mnist_cnn, hist_mnist = run_experiment(
    model=mnist_cnn,
    train_loader=mnist_train_loader,
    val_loader=mnist_val_loader,
    optimizer_name="adam",
    lr=1e-3,
    epochs=3,
    experiment_name="MNIST - SimpleCNN + Adam",
)


=== MNIST - SimpleCNN + Adam ===
Optimizer: adam, lr=0.001, epochs=3
Epoch 01 | train_loss=0.4293, train_acc=91.17% | val_loss=0.1690, val_acc=95.55%
Epoch 02 | train_loss=0.0921, train_acc=97.69% | val_loss=0.1850, val_acc=94.26%
Epoch 03 | train_loss=0.0628, train_acc=98.32% | val_loss=0.1554, val_acc=94.90%


In [ ]:
#@title 8. Quick comparison of final validation accuracies

def final_val_acc(history):
    return history["val_acc"][-1] * 100

print("Faces - SimpleCNN + SGD  :", final_val_acc(hist_cnn_sgd),  "%")
print("Faces - SimpleCNN + Adam :", final_val_acc(hist_cnn_adam), "%")
print("Faces - ResNet18 + Adam  :", final_val_acc(hist_resnet),   "%")
print("MNIST - SimpleCNN + Adam :", final_val_acc(hist_mnist),    "%")

Faces - SimpleCNN + SGD  : 98.33984375 %
Faces - SimpleCNN + Adam : 98.73046875 %
Faces - ResNet18 + Adam  : 100.0 %
MNIST - SimpleCNN + Adam : 94.89999999999999 %
